In [1]:
import json
from utils import statics


In [5]:
# 关于zhidao.dev.json (分析scores和answers)
# answer和fake_answer的match score
data_path = "src/data/dureader/process/2019/v2.0/zhidao.train.json"
scores = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        d = json.loads(line)
        if len(d["match_scores"]) > 0:
            scores.append(d["match_scores"][0])
print("about zhidao.train.json match scores: ")
statics(scores, rate=0.90, most_common=5, reversed=True)



about zhidao.train.json match scores: 
total count:  129743
mean: 0.9478705418641133, max: 1.0, min: 0.006622516556291392
most common 5: 
[(1.0, 67832), (0.6666666666666666, 1693), (0.8571428571428571, 1585), (0.8, 1427), (0.9090909090909091, 1363)]
above 0.8333333333333334 is account for 90.07114063957208%


In [ ]:
# 统计zhidao.test1.json （for only choose one para...）
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/zhidao.train.json'
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        # for doc in sample["documents"]:
        #     # count = len(doc["paragraphs"])
        #     # if count == 0:
        #     #     print(d["question_id"])
        #     for para in doc["paragraphs"]:
        #         c += len(para)
        #         # if len(para) == 0:
        #             # print(d["documents"])
        #             # print(doc["paragraphs"])
        # if c == 0:
        #     p_c += 1
        # ----------------
        # 计算最佳的para
        best_para = []
        # 取最前面三个的不为空的document
        docs = []
        for doc in sample["documents"]:
            if len(doc["segmented_paragraphs"]) != 0:
                docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 全部拼接起来
            best_para = best_para + title
            for para in paras:
                best_para = best_para + para
        before_para_len.append(len(best_para))
        # 截取一定的长度(默认500)
        best_para = best_para[: 500] if len(best_para) > 500 else best_para
        # if len(best_para) == 0:
        #     print(sample["question_id"])
        best_para_len.append(len(best_para))
        if len(best_para) == 0:
            print(sample["question_id"])
# best para len
from utils import statics
print("for choose para len statics: ")
print("before: ")
statics(before_para_len, rate=0.90, most_common=5, reversed=False)
print("after cut: ")
statics(best_para_len, rate=0.90, most_common=5, reversed=False)


In [7]:
# 统计zhidao.test1.json （for  choose multiple paras...）
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/zhidao.train.json'
train = True
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
paras_num = []  # 统计paras num分布
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        # for doc in sample["documents"]:
        #     # count = len(doc["paragraphs"])
        #     # if count == 0:
        #     #     print(d["question_id"])
        #     for para in doc["paragraphs"]:
        #         c += len(para)
        #         # if len(para) == 0:
        #             # print(d["documents"])
        #             # print(doc["paragraphs"])
        # if c == 0:
        #     p_c += 1
        # ----------------
        # 计算最佳的paras
        best_paras = []
        # 取最前面三个的不为空的document
        docs = []
        if train:
            for doc in sample["documents"]:
                if doc["is_selected"] and len(doc["segmented_paragraphs"]) != 0:
                    docs.append(doc)
        else:
            for doc in sample["documents"]:
                if len(doc["segmented_paragraphs"]) != 0:
                    docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            c_para = []
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 将每个doc的tile+4paras拼接
            c_para = c_para + title
            for para in paras:
                c_para = c_para + para
            before_para_len.append(len(c_para))
            # 截取一定的长度(默认500)
            c_para = c_para[: 500] if len(c_para) > 500 else c_para
            best_para_len.append(len(c_para))
            best_paras.append(c_para)
            if len(c_para) == 0:
                print(sample["question_id"])
        paras_num.append(len(best_paras))
# best para len
from utils import statics
print("for choose para len statics: ")
print("before: ")
statics(before_para_len, rate=0.80, most_common=5, reversed=False)
print("after cut: ")
statics(best_para_len, rate=0.90, most_common=5, reversed=False)
print("for paras num:")
statics(paras_num, rate=0.9, most_common=4, reversed=True)


for choose para len statics: 
before: 
total count:  312693
mean: 336.9170496301484, max: 22209, min: 3
most common 5: 
[(40, 1746), (35, 1729), (30, 1729), (39, 1721), (43, 1711)]
below 446 is account for 80.01362358607325%
after cut: 
total count:  312693
mean: 216.46232246964274, max: 500, min: 3
most common 5: 
[(500, 54670), (40, 1746), (35, 1729), (30, 1729), (39, 1721)]
below 500 is account for 100.0%
for paras num:
total count:  135366
mean: 2.3099818270466734, max: 3, min: 0
most common 4: 
[(3, 77754), (2, 27121), (1, 25189), (0, 5302)]
above 1 is account for 96.08321144157321%


In [10]:
# 统计search.test1.json
# 某些sample的documents中document内容可能为空
# 某些sample的paragraph可能为空，但不是全为空
# 如果使用question和para的recall来计算，仍然可能得到score为空(1825/30000多)(启发式：如果score==0， 直接取最前面的那个)
# search.test1.json中有5个sample的para全为空
import json
from utils.basic_metric import metric_max_over_ground_truths, recall
zhidao_test_1 = 'src/data/dureader/process/2019/v1.0/search.test1.json'
before_para_len = []
best_para_len = []
p_c = 0  # 统计所有para为0的sample个数
no_para_samples = []
with open(zhidao_test_1, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        c = 0  
        sample = json.loads(line)
        # number
        for doc in sample["documents"]:
            # count = len(doc["paragraphs"])
            # if count == 0:
            #     print(d["question_id"])
            for para in doc["paragraphs"]:
                c += len(para)
                # if len(para) == 0:
                    # print(d["documents"])
                    # print(doc["paragraphs"])
        if c == 0:
            p_c += 1
        # ----------------
        # 计算最佳的para
        best_para = []
        # 取最前面三个的不为空的document
        docs = []
        for doc in sample["documents"]:
            if len(doc["segmented_paragraphs"]) != 0:
                docs.append(doc)
        docs = docs[: 3]
        for doc in docs:
            title = doc["segmented_title"]
            # 取前面4个不为空的para
            paras = []
            for para in doc["segmented_paragraphs"]:
                if len(para) != 0:
                    paras.append(para)
            paras = paras[: 4]
            # 全部拼接起来
            best_para = best_para + title
            for para in paras:
                best_para = best_para + para
        before_para_len.append(len(best_para))
        # 截取一定的长度(默认500)
        best_para = best_para[: 500] if len(best_para) > 500 else best_para
        # if len(best_para) == 0:
        #     print(sample["question_id"])
        best_para_len.append(len(best_para))
        if len(best_para) == 0:
            data = {}
            print(sample["question_id"])
            data["question_id"] = sample["question_id"]
            data["question"] = sample["question"]
            data["answers"] = []
            data["question_type"] = sample["question_type"]
            data["yesno_answers"] = []
            no_para_samples.append(data)
# write no para samples to file
with open('no_para_search.json', 'w', encoding='utf-8') as f:
    for d in no_para_samples:
        d_str = json.dump(d, f, ensure_ascii=False)
        print("", file=f)
# all len(para) == 0
print("p_c: ", p_c)
# best para len
from utils import statics
print("before: ")
statics(before_para_len, rate=0.98, most_common=5)
print("after cut: ")
statics(best_para_len, rate=0.98, most_common=5)

401621


466589


391912


430591


323995


p_c:  5
before: 
mean: 462.38733333333334, max: 13584, min: 0
most common 5: 
[(330, 81), (243, 81), (263, 78), (208, 76), (303, 75)]
above 98.03666666666668% is 91
after cut: 
mean: 352.15273333333334, max: 500, min: 0
most common 5: 
[(500, 8916), (330, 81), (243, 81), (263, 78), (208, 76)]
above 98.03666666666668% is 91
